# Chapter 2 Solutions

In [25]:
import numpy as np
import pandas as pd
import scipy.misc as misc

##1. This problem concerns the ALOHA network model of Section 2.1. Feel free to use (but cite) computations already in the example.

In [2]:
class Sample():
    def __init__(self, ratio, num_samples=1000):
        self.ratio = ratio
        self.num_samples = num_samples
        
    def generate(self):
        num_ones = int(self.ratio * self.num_samples)
        num_zeros = int((1 - self.ratio) * self.num_samples)
        return np.append(np.zeros(num_zeros), np.ones(num_ones))

    def event_happens(self):
        samples = self.generate()
        np.random.shuffle(samples)
        return samples[0] == 1

class InactiveNode():
    def __init__(self, p=None, q=None):
        self.p = p
        self.q = q
        
    def maybe_become_active(self):
        if Sample(self.q).event_happens():
            return ActiveNode(p=self.p, q=self.q)
        else: 
            return InactiveNode(p=self.p, q=self.q)
        
    def is_sendable(self):
        return False
    
    def maybe_send(self, collision):
        return InactiveNode(p=self.p, q=self.q)
    
    def is_active(self):
        return False
    
    def is_inactive(self):
        return not self.is_active()



class ActiveNode():
    def __init__(self, p=None, q=None):
        self.sendable = False
        self.p = p
        self.q = q

    def maybe_become_active(self):
        return ActiveNode(p=self.p, q=self.q)
    
    def is_sendable(self):
        self.sendable = Sample(self.p).event_happens()
        return self.sendable
    
    def maybe_send(self, collision):
        if collision or not self.sendable:
            return ActiveNode(p=self.p, q=self.q)
        else:    
            return InactiveNode(p=self.p, q=self.q)

    def is_active(self):
        return True
    
    def is_inactive(self):
        return not self.is_active()


class Epoch():
    def __init__(self, nodes, num):
        self.nodes = nodes
        self.num = num

    def check_collisions(self):
        collision = True

        for node in self.nodes:
            collision = collision & node.is_sendable()
        self.collision = collision

    def maybe_send(self):
        new_nodes = []
        for node in self.nodes:
            new_nodes.append(node.maybe_send(self.collision))
        return new_nodes

    def maybe_become_active(self):
        new_nodes = []
        for node in self.nodes:
            new_nodes.append(node.maybe_become_active())
        self.nodes = new_nodes

    def num_nodes_active(self):
        count = 0
        for node in self.nodes:
            if node.is_active():
                count += 1
        return count
    
class Criteria():
    def __init__(self, criteria=None, given=lambda x: True):
        self._criteria = criteria
        self._given = given
        
    def given(self, active_count):
        return self._given(active_count)
    
    def criteria(self, active_count):
        return self._criteria(active_count)
            
def simulate_aloha(criteria=None, 
                   given=lambda x: True, 
                   num_experiments=10000,
                   num_nodes=2,
                   p=0.4,  # probability of sending as an active node
                   q=0.8): # probability of becoming active as an inactive node
    criteria = Criteria(criteria=criteria, given=given)
    
    def build_epoch(epoch=Epoch([ActiveNode(p=p, q=q), ActiveNode(p=p, q=q)], 0)):
        epoch.maybe_become_active()
        epoch.check_collisions()
        
        return Epoch(epoch.maybe_send(), epoch.num + 1)
    
    def generate_initial_number_of_nodes():
        nodes = []
        for i in range(num_nodes):
            nodes.append(ActiveNode(p=p, q=q))
        return nodes
    
    def populate_active_count(num_new_epochs):
        epoch = Epoch(generate_initial_number_of_nodes(), 0)
        active_count = [epoch.num_nodes_active()]
        for i in range(num_new_epochs):
            epoch = build_epoch(epoch=epoch)
            active_count.append(epoch.num_nodes_active())
        return active_count
    
    counts_updater = CountsUpdater(criteria)

    for experiment in range(num_experiments):
        active_count = populate_active_count(num_nodes)
        counts_updater.update(active_count)

    # needs to divide by num times that the "given" section happens, if "given" exists
    if counts_updater.denominator == 0:
        return 0.0
    else:
        return counts_updater.numerator / counts_updater.denominator

    
class CountsUpdater():
    def __init__(self, criteria):
        self.criteria = criteria
        self.numerator = 0.0
        self.denominator = 0.0
    
    def update(self, active_count):
        if self.criteria.given(active_count):
            self.denominator += 1
        if self.criteria.given(active_count) and self.criteria.criteria(active_count):
            self.numerator += 1
    

### Sanity Checks

#### Equation (2.1) in the book says $P(X_1=2) = 0.52 $

In [3]:
simulate_aloha(criteria=lambda x: x[1]==2)

0.5132

#### Equation (2.16) in the book says $P(X_1=1) = 0.48 $

In [4]:
simulate_aloha(criteria=lambda x: x[1]==1)

0.4793

#### Equation (2.17) in the book says $P(X_2=2 \mid X_1=1) = 0.41 $

In [5]:
simulate_aloha(criteria=lambda x: x[2]==2, given=lambda x: x[1]==1)

0.429322675641565

#### Equation (2.18) in the book says $P(X_1=1 \mid X_2=2) = 0.43 $

In [6]:
simulate_aloha(criteria=lambda x: x[1]==1, given=lambda x: x[2]==2)

0.4259723964868256

#### Equation (2.19) in the book says $P(X_1=2 \text{ or } X_2=2) = 0.72 $

In [7]:
simulate_aloha(criteria=lambda x: x[1] == 2 or x[2] == 2)

0.726

### (a) Calculate $P(X_{1} = 2 \text{ and } X_{2} = 1)$, for the same values of $p$ and $q$ in the examples.

\begin{equation}
  P(X_{1} = 2 \text{ and } X_{2} = 1) = P(X_{2} = 1 \mid X_{1} = 2) P(X_{1} = 2)
\end{equation}

Formally speaking:

\begin{equation}
\begin{split}
  P(X_{1} = 2) &= \sum_{i=0}^{2}{P(X_{1} = 2 \mid X_{0} = i) P(X_{0} = i)} \\
  &= P(X_{1} = 2 \mid X_{0} = 0) P(X_{0} = 0) \\
  &+  P(X_{1} = 2 \mid X_{0} = 1) P(X_{0} = 1) \\
  &+ P(X_{1} = 2 \mid X_{0} = 2) P(X_{0} = 2)
\end{split}
\end{equation}


However, since it was given that both nodes were active in the beginning (i.e.
$P(X_{0} = 2) = 1$), we know that $P(X_{0} = 0) = P(X_{0} = 1) = 0$. Thus, the
statement above simplifies to:

\begin{equation}
\begin{aligned}
  P(X_{1} = 2) &= P(X_{1} = 2 \mid X_{0} = 2)
\end{aligned}
\end{equation}

$P(X_{1} = 2 \mid X_{0} = 2)$ could only happen two ways: either both
active nodes send information (and hence create a collision), or both active
nodes don't send anything at all.

\begin{equation}
  \begin{aligned}
    P(X_{1} = 2) &= P(X_{1} = 2 \mid X_{0} = 2) \\
    &= P(C_{1} = 1 \text{ and } C_{2} = 1 \text{ or } C_{1} = 0 \text{ and } C_{2} = 0) \\
                 &= P(C_{1} = 1 \text{ and } C_{2} = 1) + P(C_{1} = 0 \text{ and } C_{2} = 0) \\
    &= p^2 + (1-p)^2 \\
    &= (0.4)^2 + (1-0.4)^2 \\
    &= 0.16 + 0.36 \\
    &= 0.52 \\
  \end{aligned}
\end{equation}

\begin{equation}
  \begin{aligned}
    P(X_{2} = 1 \mid X_{1} = 2) &= P(C_{1} = 1 \text{ and } C_{2} = 0 \text{ or } C_{1} = 0 \text{ and } C_{2} = 1) \\
    &= P(C_{1} = 1 \text{ and } C_{2} = 0) + P(C_{1} = 0 \text{ and } C_{2} =  1) \\
    &= p(1-p) + p(1-p) \\
    &= 2p(1-p) \\
    &= 2(0.4)(1-(0.4)) \\
    &= 0.48\\
  \end{aligned}
\end{equation}

Therefore:

\begin{equation}
  \begin{aligned}
    P(X_{1} = 2 \text{ and } X_{2} = 1) &= P(X_{2} = 1 \mid X_{1} = 2) P(X_{1} = 2) \\
    &= 0.48 \times 0.52 \\
    &= 0.2496
  \end{aligned}
\end{equation}




In [8]:
simulate_aloha(lambda x: x[1] == 2 and x[2] == 1)

0.2473

### (b) Find $P(X_{2} = 0)$


\begin{equation}
  \begin{aligned}
    P(X_2 = 0) &= \sum_{i=0}^{2} P(X_2 = 0 \text{ and } X_1 = i) \\
    &= \sum_{i=0}^{2} P(X_2 = 0 \mid X_1 = i)P(X_1 = i) \\
    &= P(X_2 = 0 \mid X_1 = 0)P(X_1 = 0) \\
    &\quad + P(X_2 = 0 \mid X_1 = 1)P(X_1 = 1) \\
    &\quad + P(X_2 = 0 \mid X_1 = 2)P(X_1 = 2) \\
  \end{aligned}
\end{equation}

Let's start with $P(X_1 = 0)$. We know that it is equivalent to:

\begin{equation}
  \begin{aligned}
    P(X_1 = 0) &= P(X_1 = 0 \text{ and } X_0 = 2) \\
    &= P(X_1 = 0 \mid X_0 = 2)P(X_0 = 2) \\
    &= P(X_1 = 0 \mid X_0 = 2) \\
  \end{aligned}
\end{equation}

It is impossible for two active nodes to both become inactive for the next
epoch, so $P(X_1 = 0) = P(X_1 = 0 \mid X_0 = 2) = 0$.

Next we look at $P(X_1 = 1)$:

\begin{equation}
  \begin{aligned}
    P(X_1 = 1) &= P(X_1 = 1 \text{ and } X_0 = 2) \\
    &= P(X_1 = 1 \mid X_0 = 2)P(X_0 = 2) \\
    &= P(X_1 = 1 \mid X_0 = 2) \\
  \end{aligned}
\end{equation}

This could only happen two ways: either the first node sends and the second
node does not, or the second node sends and the first node does not. For active
nodes, let $S_i=k, i \in \{0,1\}$ and $k \in \{0,1\}$ where, for node $i$,
$S_i=0$ is the event of not sending and $S_i=1$ is the event of sending:

\begin{equation}
  \begin{aligned}
    P(X_1 = 1) &= P(X_1 = 1 \mid X_0 = 2) \\
    &= P(S_1 = 1 \text{ and } S_2 = 0 \text{ or } S_1 = 0 \text{ and } S_2 = 1) \\
    &= P(S_1 = 1 \text{ and } S_2 = 0) + P(S_1 = 0 \text{ and } S_2 = 1) \\
    &= p(1-p) + p(1-p) \\
    &= 2p(1-p) \\
    &= 2(0.4)(1-0.4) \\
    &= 0.48
  \end{aligned}
\end{equation}

Now let's look at $P(X_2=0 \mid X_1=1)$. The only way this could happen is when
the active node sends while the other node stays inactive:

\begin{equation}
  \begin{aligned}
    P(X_2=0 \mid X_1=1) &= p(1-q) \\
    &= (0.4)(1-(0.8)) \\
    &= 0.08
  \end{aligned}
\end{equation}

Now we consider $P(X_2=0 \mid X_1=2)$. Again it is impossible for two active
nodes to become inactive for the next epoch, so $P(X_2=0 \mid X_1=2) = 0$.

Therefore, $P(X_2=0)$ simplifies to the following:

\begin{equation}
  \begin{aligned}
    P(X_2=0) &= P(X_2 = 0 \mid X_1 = 0)P(X_1 = 0) \\
    &\quad + P(X_2 = 0 \mid X_1 = 1)P(X_1 = 1) \\
    &\quad + P(X_2 = 0 \mid X_1 = 2)P(X_1 = 2) \\
    &= P(X_2 = 0 \mid X_1 = 0) \times 0 \\
    &\quad + 0.08 \times 0.48 \\
    &\quad + 0 \times P(X_1 = 2) \\
    &= 0.0384
  \end{aligned}
\end{equation}





In [9]:
simulate_aloha(lambda x: x[2]==0)

0.0381

### (c) Find $P(X_{1} = 1 \mid X_{2} = 1)$

\begin{equation}
  \begin{aligned}
    P(X_{1} = 1 \mid X_{2} = 1) &= \frac{P(X_1=1 \text{ and } X_2=1)}{P(X_2 = 1)} \\
    &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{P(X_2 = 1)} \\
    &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{\sum_{i=0}^{2}P(X_2 = 1, X_1=i)} \\
    &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{\sum_{i=0}^{2}P(X_2 = 1 \mid X_1=i)P(X_1=i)} \\
  \end{aligned}
\end{equation}

The denominator expands to the following:

\begin{equation}
  \begin{aligned}
    \sum_{i=0}^{2}P(X_2 = 1 \mid X_1=i)P(X_1=i) &= P(X_2 = 1 \mid X_1=0)P(X_1=0) \\
    &\quad + P(X_2 = 1 \mid X_1=1)P(X_1=1) \\
    &\quad + P(X_2 = 1 \mid X_1=2)P(X_1=2) \\
  \end{aligned}
\end{equation}

We've already established that $P(X_1=0) = 0$, $P(X_1=1)=0.48$, $P(X_1=2) =
0.52$. Since, $P(X_1=0) = 0$, we know that $P(X_2=1 \mid X_1=0)$ cannot
happen. Thus, what we need to calculate are the following: $P(X_2=1
\mid X_1=1)$ and $P(X_2=1 \mid X_1=2)$.

Let's start with $P(X_2=1 \mid X_1=1)$. Three possibilities:

* Active node successfully sends; inactive node becomes active and does not send
* Active node does not attempt to send; inactive node does not become active
* Active node does not attempt to send; inactive node becomes active and successfully sends

\begin{equation}
  \begin{aligned}
    P(X_2=1 \mid X_1=1) &= (1-p)qp + (1-p)(1-q) + pq(1-p) \\
    &= (1-0.4)(0.8)(0.4) + (1-0.4)(1-0.8) + (0.4(0.8(1-0.4))) \\
    &= 0.192 + 0.12 + 0.192 \\
    &= 0.504
  \end{aligned}
\end{equation}

Next we look at $P(X_2=1 \mid X_1=2)$. The first active node sends a message
and becomes inactive while the second active node refrains from sending, or the
second active node sends a message and becomes inactive while the first active
node refrains.

\begin{equation}
  \begin{aligned}
    P(X_2=1 \mid X_1=2) &= P(A_1=1 \text{ and } A_2=0 \text{ or } A_1=0 \text{ and } A_2=1) \\
    &= P(A_1=1 \text{ and } A_2=0) + P(A_1=0 \text{ and } A_2=1) \\
    &= p(1-p) + (1-p)p \\
    &= 2p(1-p) \\
    &= 2(0.4)(1-0.4) \\
    &= 0.48
  \end{aligned}
\end{equation}

Therefore, $P(X_1=1 \mid X_2=1)$ reduces to the following:

\begin{equation}
  \begin{aligned}
    P(X_1=1 \mid X_2=1) &= \frac{P(X_2=1 \mid X_1=1)P(X_1=1) }{\sum_{i=0}^{2}P(X_2 = 1 \mid X_1=i)P(X_1=i)} \\
    &= \frac{0.504 \times 0.48}{0 \times 0 + 0.504 \times 0.48 + 0.48 \times 0.52}\\
    &= \frac{0.24192}{0 + 0.24192 + 0.2496} \\
    &= 0.4921875 \\
  \end{aligned}
\end{equation}




In [10]:
simulate_aloha(criteria=lambda x: x[1]==1, given=lambda x: x[2]==1)

0.4897959183673469

## 2.  Urn I contains three blue marbles and three yellow ones, while Urn II contains five and seven of these colors. We draw a marble at random from Urn I and place it in Urn II. We then draw a marble at random from Urn II.


In [18]:
def simulate_urn_drawing(criteria=None, given=None, num_experiments=10000):
    draws = ['placeholder']
    experiments = []
    first_draws = []
    second_draws = []
    
    def urn_1(): 
        return ['B','B','B','Y','Y','Y']
    def urn_2():
        return ['B','B','B','B','B','Y','Y','Y','Y','Y','Y','Y']

    for i in range(num_experiments):
        first_draw = np.random.permutation(urn_1())[0]
        urn_2_copy = urn_2()
        urn_2_copy.append(first_draw)
        second_draw = np.random.permutation(urn_2_copy)[0]
        
        first_draws.append(first_draw)
        second_draws.append(second_draw)
        
    df = pd.DataFrame({'first_draw': first_draws, 'second_draw': second_draws})

    if given==None:
        return df[criteria(df)].shape[0] / float(num_experiments)
    else:
        meeting_what_is_given = df[given(df)]
        return meeting_what_is_given[criteria(meeting_what_is_given)].shape[0] \
                / float(meeting_what_is_given.shape[0])
    

### a. Find $P(\text{second marble drawn is blue})$

Let $M_i=k$ represent the marble drawn for the $i$-th drawing where $k \in
\{B, Y\}$. $B$ stands for 'Blue' and $Y$ stands for 'yellow'.

\begin{equation}
  \begin{aligned}
    P(M_2=B) &= P(M_2=B, M_1=B) + P(M_2=B, M_1=Y)\\
    &= P(M_2=B \mid M_1=B)P(M_1=B) + P(M_2=B \mid M_1=Y)P(M_1=Y)\\
    &= \frac{6}{13}(0.5) + \frac{5}{13}(0.5)\\
    &= 0.4230769231
  \end{aligned}
\end{equation}


In [19]:
simulate_urn_drawing(criteria=lambda df: df['second_draw']=='B')

0.4241

### a. Find $P(\text{first marble drawn is blue} \mid \text{second marble drawn is blue})$

\begin{equation}
  \begin{aligned}
    P(M_1=B \mid M_2=B) &= \frac{P(M_2=B, M_1=B)}{P(M_2=B)}\\
    &= \frac{P(M_2=B \mid M_1=B)P(M_1=B)}{P(M_2=B)}\\
    &= \frac{(6/13)(1/2)}{11/26}\\
    &= 0.\overline{54}
  \end{aligned}
\end{equation}



In [20]:
simulate_urn_drawing(criteria=lambda df: df['first_draw']=='B', given=lambda df: df['second_draw']=='B')

0.5454981992797119

## 3. Consider the example of association rules in Section 2.15.4. How many two-antecedent, two- consequent rules are possible from 20 items? Express your answer in terms of combinatorial (“n choose k”) symbols.

\begin{equation}
  \begin{aligned}
    {20 \choose 2} {18 \choose 2} &= \frac{20!}{2! \times 18!} \times \frac{18!}{2!16!} \\
    &= \frac{20 \times 19 \times 18 \times 17 \times 16!}{4 \times 16!} \\
    &= 5 \times 19 \times 18 \times 17 \\
    &= 29\,070
  \end{aligned}
\end{equation}

In [28]:
misc.comb(20, 2) * misc.comb(18,2)

29070.0

### Simplification of the problem: How many two-antecedent, two-consequent rules are possible from 5 items?

To get better confidence about the solution for 20 items above, I simplified the problem to 5 items. I expect 30 different configurations:

\begin{equation}
  \begin{aligned}
    {5 \choose 2} {3 \choose 2} &= \frac{5!}{2! \times 3!} \times \frac{3!}{2!1!} \\
    &= \frac{5 \times 4 \times 3 \times 2}{4} \\
    &= 5 \times 3 \times 2 \\
    &= 30
  \end{aligned}
\end{equation}

### Here are all the possible configurations:

![How many two-antecedent and two-consequent rules are possible from 5 items?](images/2-precedents-2-antecedents-5-choices.jpeg "Title")

## 4. Suppose 20% of all C++ programs have at least one major bug. Out of five programs, what is the probability that exactly two of them have a major bug?

\begin{equation}
  \begin{aligned}
    P(\text{bug}) &= 0.2 \\
    P(\text{no bug}) &= 0.8 \\
    P(\text{bug})^2 \times P(\text{no bug})^3 \times {5 \choose 2} &= 0.2^2 \times 0.8^3 \frac{5!}{2!\times3!}\\
    &= 0.04 \times 0.512 \times 10.0\\
    &= 0.2048 \\
  \end{aligned}
\end{equation}

In [46]:
def bug_percentage(num_programs=5, exactly_have_bug=2, num_experiments=10000, prob_bug=0.2):
    ones = np.ones(int(prob_bug * 10))
    zeros = np.zeros(int((1.0-prob_bug) * 10))
    
    vals = np.append(ones, zeros)
   
    count = 0
    for i in range(num_experiments):
        
        result = []
        for x in range(num_programs):
            val = np.random.permutation(vals)[0]
            result.append(val)
            
        if np.array(result).sum() == exactly_have_bug:
            count += 1
    
    return count / float(num_experiments)
        
bug_percentage()

0.2047

## 5. Assume the ALOHA network model as in Section 2.1, i.e. $m = 2$ and $X_0 = 2$, but with general values for $p$ and $q$. Find the probability that a new message is created during epoch 2.

The term "created" is ambiguous to me. It could mean "created" in a sense that the message is actually sent, or it could mean "created" in a sense that an inactive node generates a new message to send (which may, or may not, actually be sent at some epoch of interest). I'm going to assume that it means the latter.

Let $G_i$ be the number of messages created at the $i$-th epoch. The probability that a message gets created during the second epoch, then, is the probability that one message or two messages get created during the second epoch:

\begin{equation}
  \begin{aligned}
    P(G_2=1 \text{ or } G_2=2) &= P((G_2=1 \text{ or } G_2=2) \text{ and } X_1=0) \\
    &\quad + P((G_2=1 \text{ or } G_2=2) \text{ and } X_1=1) \\
    &\quad + P((G_2=1 \text{ or } G_2=2) \text{ and } X_1=2) \\
  \end{aligned}
\end{equation}

We know from previous exercises, however, that $P(X_1=0)$ is impossible. We also know that given $X_1=2$, we cannot generate new messages during the second epoch -- both nodes were already active then, which means that during the second epoch, we cannot create new messages. Thus, the above equation simplifies to:

\begin{equation}
  \begin{aligned}
    P(G_2=1 \text{ or } G_2=2) &= P((G_2=1 \text{ or } G_2=2) \text{ and } X_1=1) \\
  \end{aligned}
\end{equation}

When one node is already active during the first epoch, we know we can only create one more message (not two). Thus, the above equation simplifies to something even simpler:

\begin{equation}
  \begin{aligned}
    P(G_2=1 \text{ or } G_2=2) &= P(G_2=1 \text{ and } X_1=1) \\
    &= P(G_2=1 \mid X_1=1)P(X_1=1) \\
  \end{aligned}
\end{equation}

We've already established in previous questions that $P(X_1=1) = p^2 + (1-p)^2$. When $X_1=1$, the only way for $G_2=1$ to happen is for the inactive node to become active, which has a probability of $q$.

Therefore, the whole thing simplifies to the following:

\begin{equation}
  \begin{aligned}
    P(G_2=1 \text{ or } G_2=2) &= q(p^2 + (1-p)^2)
  \end{aligned}
\end{equation}





# 